<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Go to Google Drive and upload video files to the folder

https://drive.google.com/drive/

In [ ]:
#@title ##**Connecting to Google Drive** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ##**Enter foder name** { display-mode: "form" }

import os

folder_name = input('Enter the name of the folder on Google Drive where the video files are located: ')



In [ ]:
#@title ##**Install** { display-mode: "form" }
!apt-get install ffmpeg
!pip install moviepy

In [ ]:
#@title ##**Run** { display-mode: "form" }
import os
import subprocess
from moviepy.editor import VideoFileClip

# Define the path to the folder with video files
folder_path = os.path.join('/content/drive/MyDrive', folder_name)

# Check if the folder exists
if not os.path.exists(folder_path):
    raise FileNotFoundError(f"The folder '{folder_name}' does not exist in your Google Drive.")

# Get a list of video files in the folder, sorted alphabetically
video_extensions = ['.mp4', '.mov', '.avi', '.mkv']
video_files = [f for f in sorted(os.listdir(folder_path)) if os.path.splitext(f)[1].lower() in video_extensions]

if not video_files:
    raise FileNotFoundError("No video files found in the specified folder.")

print("Video files to be processed:")
for idx, video_file in enumerate(video_files, 1):
    print(f"{idx}. {video_file}")

# Determine the output resolution based on the video with the largest width
max_width = 0
output_width = 0
output_height = 0
for video_file in video_files:
    video_path = os.path.join(folder_path, video_file)
    try:
        clip = VideoFileClip(video_path)
        width, height = clip.size
        if width > max_width:
            max_width = width
            output_width = width
            output_height = height
        clip.close()
    except Exception as e:
        print(f"Error processing file {video_file}: {e}")
        raise

print(f"\nOutput resolution determined: {output_width}x{output_height}")

# Create a temporary directory to store processed videos
temp_folder = '/content/temp_videos'
os.makedirs(temp_folder, exist_ok=True)

# Process each video file
print("\nProcessing video files:")
for idx, video_file in enumerate(video_files, 1):
    input_video_path = os.path.join(folder_path, video_file)
    temp_video_path = os.path.join(temp_folder, f'part{idx:03d}.mp4')

    print(f"Processing {idx}/{len(video_files)}: {video_file}")

    # Get the original dimensions
    clip = VideoFileClip(input_video_path)
    width, height = clip.size
    clip.close()

    # Calculate the scale factor to match the output height
    scale_factor = output_height / height
    new_width = int(width * scale_factor)
    new_height = output_height

    # Prepare the ffmpeg video filter (vf) options
    if new_width < output_width:
        # Pad black bars on the sides
        pad_width = (output_width - new_width) // 2
        vf_filter = f"scale={new_width}:{new_height},pad={output_width}:{output_height}:{pad_width}:0:black"
    elif new_width > output_width:
        # Crop the sides
        crop_start_x = (new_width - output_width) // 2
        vf_filter = f"scale={new_width}:{new_height},crop={output_width}:{output_height}:{crop_start_x}:0"
    else:
        # No padding or cropping needed
        vf_filter = f"scale={new_width}:{new_height}"

    # Build the ffmpeg command
    command = [
        'ffmpeg', '-y', '-i', input_video_path,
        '-vf', vf_filter,
        '-c:v', 'libx264', '-preset', 'fast', '-crf', '18',
        '-c:a', 'aac', '-b:a', '192k',
        temp_video_path
    ]

    # Run the ffmpeg command
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        error_message = result.stderr.decode()
        print(f"Error processing file {video_file}:\n{error_message}")
        raise RuntimeError(f"FFmpeg error: {error_message}")
    else:
        print(f"Successfully processed {video_file}")

print("\nAll video files have been processed.")

# Create a text file listing the processed videos for concatenation
concat_list_path = '/content/concat_list.txt'
with open(concat_list_path, 'w') as f:
    for idx in range(1, len(video_files) + 1):
        temp_video_path = os.path.join(temp_folder, f'part{idx:03d}.mp4')
        f.write(f"file '{temp_video_path}'\n")

# Merge the videos
print("\nMerging video files...")
output_video_path = '/content/drive/MyDrive/merged_video.mp4'

# Use ffmpeg to concatenate videos
merge_command = [
    'ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', concat_list_path,
    '-c:v', 'copy', '-c:a', 'copy',
    output_video_path
]

result = subprocess.run(merge_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode != 0:
    error_message = result.stderr.decode()
    print(f"Error merging videos:\n{error_message}")
    # If copying codecs fails due to codec differences, re-encode during merging
    print("Attempting to merge with re-encoding...")
    merge_command = [
        'ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', concat_list_path,
        '-c:v', 'libx264', '-preset', 'fast', '-crf', '18',
        '-c:a', 'aac', '-b:a', '192k',
        output_video_path
    ]
    result = subprocess.run(merge_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        error_message = result.stderr.decode()
        print(f"Error merging videos with re-encoding:\n{error_message}")
        raise RuntimeError(f"FFmpeg merge error: {error_message}")
    else:
        print("Videos merged successfully with re-encoding.")
else:
    print("Videos merged successfully.")

print(f"\nMerged video saved to Google Drive as 'merged_video.mp4'")

In [ ]:
#@title ##**Download video** { display-mode: "form" }
import shutil
shutil.copy(output_video_path, '/content/merged_video.mp4')

from google.colab import files
files.download('/content/merged_video.mp4')